# e_SVC
----

Written in the Python 3.7.9 Environment with the following package versions

    * joblib 1.0.1
    * numpy 1.19.5
    * pandas 1.3.1
    * scikit-learn 0.24.2
    * tensorflow 2.5.0

By Nicole Lund 

This Jupyter Notebook tunes a SVC model for Exoplanet classification from Kepler Exoplanet study data.

Column descriptions can be found at https://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html 

**Source Data**

The source data used was provided by University of Arizona's Data Analytics homework assignment. Their data was derived from https://www.kaggle.com/nasa/kepler-exoplanet-search-results?select=cumulative.csv

The full data set was released by NASA at
https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=koi

In [1]:
# Import Dependencies

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt

# Data manipulation
import numpy as np
import pandas as pd
from statistics import mean
from operator import itemgetter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

# Parameter Selection
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Model Development
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Model Metrics
from sklearn.metrics import classification_report

# Save/load files
from tensorflow.keras.models import load_model
import joblib

# # Ignore deprecation warnings
# import warnings
# warnings.simplefilter('ignore', FutureWarning)

In [2]:
# Set the seed value for the notebook, so the results are reproducible
from numpy.random import seed
seed(1)

# Read the CSV and Perform Basic Data Cleaning

In [3]:
# Import data
df = pd.read_csv("../b_source_data/exoplanet_data.csv")
# print(df.info())

# Drop columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop rows containing null values
df = df.dropna()

# Display data info
print(df.info())
print(df.head())
print(df.koi_disposition.unique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6991 entries, 0 to 6990
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   koi_disposition    6991 non-null   object 
 1   koi_fpflag_nt      6991 non-null   int64  
 2   koi_fpflag_ss      6991 non-null   int64  
 3   koi_fpflag_co      6991 non-null   int64  
 4   koi_fpflag_ec      6991 non-null   int64  
 5   koi_period         6991 non-null   float64
 6   koi_period_err1    6991 non-null   float64
 7   koi_period_err2    6991 non-null   float64
 8   koi_time0bk        6991 non-null   float64
 9   koi_time0bk_err1   6991 non-null   float64
 10  koi_time0bk_err2   6991 non-null   float64
 11  koi_impact         6991 non-null   float64
 12  koi_impact_err1    6991 non-null   float64
 13  koi_impact_err2    6991 non-null   float64
 14  koi_duration       6991 non-null   float64
 15  koi_duration_err1  6991 non-null   float64
 16  koi_duration_err2  6991 

In [4]:
# Rename "FALSE POSITIVE" disposition values
df.koi_disposition = df.koi_disposition.str.replace(' ','_')
print(df.koi_disposition.unique())

['CONFIRMED' 'FALSE_POSITIVE' 'CANDIDATE']


# Select features

In [5]:
# Split dataframe into X and y

# Select features to analyze in X
select_option = 1

if select_option == 1:
    # Option 1: Choose all features
    X = df.drop("koi_disposition", axis=1)

elif select_option == 2:
    # Option 2: Choose all features that are not associated with error measurements
    X = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration','koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_slogg', 'koi_srad', 'ra', 'dec', 'koi_kepmag']]

elif select_option == 3:
    # Option 3: Choose features from Decision Tree and Random Forest assessment.
    tree_features = ['koi_fpflag_nt', 'koi_fpflag_co', 'koi_fpflag_ss', 'koi_model_snr']
    forest_features = ['koi_fpflag_co', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_model_snr', 'koi_prad']
    X = df[set(tree_features + forest_features)]

# Define y
y = df["koi_disposition"]

print(X.shape, y.shape)

(6991, 40) (6991,)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
# Split X and y into training and testing groups
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [7]:
# Display training data
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
4954,0,0,0,0,97.942437,0.001183,-0.001183,157.97325,0.00888,-0.00888,...,-198,3.806,0.445,-0.194,2.284,0.726,-1.090,288.53769,51.280384,11.366
4235,0,0,1,1,16.609490,0.000141,-0.000141,147.61311,0.00715,-0.00715,...,-210,4.395,0.105,-0.195,1.050,0.323,-0.162,300.43979,43.930729,14.926
848,0,0,0,0,32.275385,0.000541,-0.000541,149.04600,0.01400,-0.01400,...,-103,4.123,0.182,-0.098,1.483,0.230,-0.316,298.29495,48.775459,14.167
2874,0,1,1,1,0.787120,0.000002,-0.000002,131.73939,0.00190,-0.00190,...,-201,4.481,0.150,-0.100,0.787,0.093,-0.103,295.28110,40.132858,14.964
3016,0,0,0,0,6.963527,0.000039,-0.000039,133.01717,0.00435,-0.00435,...,-201,3.978,0.458,-0.153,1.686,0.413,-0.709,296.10406,42.342590,13.128


# Pre-processing

In [8]:
# Scale the data with MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# One-Hot-Encode the y data

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [10]:
print('Unique KOI Disposition Values')
print(y.unique())
print('-----------')
print('Sample KOI Disposition Values and Encoding')
print(y_test[:5])
print(y_test_categorical[:5])

Unique KOI Disposition Values
['CONFIRMED' 'FALSE_POSITIVE' 'CANDIDATE']
-----------
Sample KOI Disposition Values and Encoding
4982    FALSE_POSITIVE
4866         CANDIDATE
2934    FALSE_POSITIVE
5007    FALSE_POSITIVE
3869    FALSE_POSITIVE
Name: koi_disposition, dtype: object
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


# Create and Train the Model - SVM



In [11]:
# Create model
model = SVC(C=10, gamma=0.0001, kernel='linear')

# Train model
model.fit(X_train_scaled, y_train)

SVC(C=10, gamma=0.0001, kernel='linear')

In [12]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.880032699775189
Testing Data Score: 0.8746425166825548


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [13]:
# Create the GridSearchCV model
param_grid = [{'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01], 'kernel': ['linear']},
              {'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01], 'kernel': ['rbf']}]
grid = GridSearchCV(model, param_grid, verbose=3)

In [14]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END ..C=1, gamma=0.0001, kernel=linear;, score=0.868 total time=   0.1s
[CV 2/5] END ..C=1, gamma=0.0001, kernel=linear;, score=0.834 total time=   0.1s
[CV 3/5] END ..C=1, gamma=0.0001, kernel=linear;, score=0.858 total time=   0.1s
[CV 4/5] END ..C=1, gamma=0.0001, kernel=linear;, score=0.848 total time=   0.1s
[CV 5/5] END ..C=1, gamma=0.0001, kernel=linear;, score=0.838 total time=   0.1s
[CV 1/5] END ...C=1, gamma=0.001, kernel=linear;, score=0.868 total time=   0.1s
[CV 2/5] END ...C=1, gamma=0.001, kernel=linear;, score=0.834 total time=   0.1s
[CV 3/5] END ...C=1, gamma=0.001, kernel=linear;, score=0.858 total time=   0.1s
[CV 4/5] END ...C=1, gamma=0.001, kernel=linear;, score=0.848 total time=   0.1s
[CV 5/5] END ...C=1, gamma=0.001, kernel=linear;, score=0.838 total time=   0.1s
[CV 1/5] END ....C=1, gamma=0.01, kernel=linear;, score=0.868 total time=   0.1s
[CV 2/5] END ....C=1, gamma=0.01, kernel=linear;

GridSearchCV(estimator=SVC(C=10, gamma=0.0001, kernel='linear'),
             param_grid=[{'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01],
                          'kernel': ['linear']},
                         {'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01],
                          'kernel': ['rbf']}],
             verbose=3)

In [15]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}
0.8753276892451083


In [16]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)

In [17]:
print(classification_report(y_test, predictions,
                            target_names=y.unique()))

                precision    recall  f1-score   support

     CONFIRMED       0.81      0.65      0.72       504
FALSE_POSITIVE       0.74      0.85      0.79       574
     CANDIDATE       0.98      1.00      0.99      1020

      accuracy                           0.87      2098
     macro avg       0.85      0.83      0.84      2098
  weighted avg       0.88      0.87      0.87      2098



# Option 1: Model Results when using all features
* Training Data Score: 0.880032699775189
* Testing Data Score: 0.8746425166825548
* Tuned Parameters: {'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}
* Tuning Score: 0.8753276892451083
* Tuned Classification Report:

                        precision    recall  f1-score   support

        CONFIRMED       0.81      0.65      0.72       504
        FALSE POSITIVE  0.74      0.85      0.79       574
        CANDIDATE       0.98      1.00      0.99      1020

        accuracy                            0.87      2098
        macro avg       0.85      0.83      0.84      2098
        weighted avg    0.88      0.87      0.87      2098

# Option 2: Model Results when using all features not associated with error measurements
* Training Data Score: 0.8187206212957286
* Testing Data Score: 0.8060057197330791
* Tuned Parameters: {'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}
* Tuning Score: 0.8160626740277943
* Tuned Classification Report:

                    precision    recall  f1-score   support

        CONFIRMED       0.61      0.58      0.60       504
        FALSE POSITIVE  0.65      0.66      0.66       574
        CANDIDATE       0.98      1.00      0.99      1020

        accuracy                           0.81      2098
        macro avg       0.75      0.75      0.75      2098
        weighted avg    0.80      0.81      0.80      2098

# Option 3: Model Results when using selected features from Decision Tree and Random Forest Classifiers
* Training Data Score: 0.7422848967913346
* Testing Data Score: 0.7149666348903718
* Tuned Parameters: {'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}
* Tuning Score: 0.7467815955097957
* Tuned Classification Report:

                        precision    recall  f1-score   support

        CONFIRMED       0.46      0.97      0.62       504
        FALSE POSITIVE  0.68      0.04      0.08       574
        CANDIDATE       0.98      0.97      0.98      1020

        accuracy                            0.71      2098
        macro avg       0.71      0.66      0.56      2098
        weighted avg    0.77      0.71      0.65      2098

# Save the Model

Option 1 was chosen as the model to save because it yielded the best accuracy and candidate recall of all 3 input options.

In [18]:
# Save the model results
joblib.dump(model, './e_SVC_model.sav')
joblib.dump(grid, './e_SVC_grid.sav')

['./e_SVC_grid.sav']

# Model Discussion

The option 1 model score using the SVM method is one of the best for predicting exoplanet observations. The Neural Network model performs similarly, but hyperparameter tuning takes significantly more time to its detriment.